In [36]:
import pomegranate
import logging
logging.basicConfig(level=logging.INFO)

## Task1: Monty Hall Problem
![task1](figures/task1.png)
As we know:

$P(guest = A) = P(guest = B) = P(guest = C) = \frac{1}{3}$
$P(prize = A) = P(prize = B) = P(prize = C) = \frac{1}{3}$

When $guest$ is chosen, then $monty$ will be randomly chosen from $\lbrace A, B, C\rbrace - \lbrace guest, prize\rbrace$

The question is:
$$P(guest = A, prize = C, monty = B)$$
$$P(guest = A, prize = C, monty = A)$$

In [3]:
guest = pomegranate.DiscreteDistribution(
    {
        'A': 1 / 3,
        'B': 1 / 3,
        'C': 1 / 3
    }
)

In [4]:
prize = pomegranate.DiscreteDistribution(
    {
        'A': 1 / 3,
        'B': 1 / 3,
        'C': 1 / 3
    }
)

In [12]:
S = ['A', 'B', 'C']
f = lambda x, y, z: \
        (0.0 if z == x else 0.5) \
        if x == y else \
        (0.0 if z == x or z == y else 1.0)
monty = [[i, j, k, f(i, j, k)] for i in S for j in S for k in S]
monty = pomegranate.ConditionalProbabilityTable(
    monty, [guest, prize])

In [14]:
s1 = pomegranate.State(guest, name="guest")
s2 = pomegranate.State(prize, name="prize")
s3 = pomegranate.State(monty, name="monty")

model = pomegranate.BayesianNetwork("Monty Hall Problem")
model.add_states(s1, s2, s3)
model.add_transition(s1, s3)
model.add_transition(s2, s3)
model.bake()

In [16]:
model.probability(['A', 'C', 'B'])

0.11111111111111109

In [17]:
model.probability(['A', 'C', 'A'])

0.0

## Burglary

![task2](figures/task2.png)

**Question:**
1. $P(JohnCalls, MaryCalls)$
2. $P(Burglary, Earthquake, Alarm, JohnCalls, MaryCalls)$
3. $P(Alarm | JohnCalls, MaryCalls)$
4. $P(JohnCalls, \neg MaryCalls | \neg Burglary)$

In [31]:
burglary = pomegranate.DiscreteDistribution(
    {
        True: 0.001,
        False: 0.999
    }
)

earthquake = pomegranate.DiscreteDistribution(
    {
        True: 0.002,
        False: 0.998
    }
)

S = {True: 0, False: 1}

alarm = [0.95, 0.94, 0.29, 0.001]
f = lambda x, y, z: \
        alarm[2 * S[x] + S[y]] if z else 1-alarm[2 * S[x] + S[y]]
alarm = [[i, j, k, f(i, j, k)] for i in S for j in S for k in S]
alarm = pomegranate.ConditionalProbabilityTable(
    alarm, [burglary, earthquake])

johncalls = [0.90, 0.05]
f = lambda x, y: \
        johncalls[S[x]] if y else 1-johncalls[S[x]]
johncalls = [[i, j, f(i, j)] for i in S for j in S]
johncalls = pomegranate.ConditionalProbabilityTable(
    johncalls, [alarm])

marycalls = [0.70, 0.01]
f = lambda x, y: \
        marycalls[S[x]] if y else 1-marycalls[S[x]]
marycalls = [[i, j, f(i, j)] for i in S for j in S]
marycalls = pomegranate.ConditionalProbabilityTable(
    marycalls, [alarm])

In [33]:
s1 = pomegranate.State(burglary, name="Burglary")
s2 = pomegranate.State(earthquake, name="Earthquake")
s3 = pomegranate.State(alarm, name="Alarm")
s4 = pomegranate.State(johncalls, name="JohnCalls")
s5 = pomegranate.State(marycalls, name="MaryCalls")

model = pomegranate.BayesianNetwork("Burglary")
model.add_states(s1, s2, s3, s4, s5)
model.add_transition(s1, s3)
model.add_transition(s2, s3)
model.add_transition(s3, s4)
model.add_transition(s3, s5)
model.bake()

In [69]:
def get_probability(model, target, condition):
    ret = 1.0
    for i, v in enumerate(target):
        if v is None:
            continue
        tmp = model.predict_proba(condition)
        ret *= tmp[i].parameters[0][v]
        condition[i] = v
    return ret

In [70]:
get_probability(
    model, 
    [None, None, None, True, True], 
    [None, None, None, None, None]
)

0.0020841002390004624

In [71]:
get_probability(
    model, 
    [True, True, True, True, True], 
    [None, None, None, None, None]
)

1.1970000000007947e-06

In [72]:
get_probability(
    model, 
    [None, None, True, None, None], 
    [None, None, None, True, True]
)

0.7606920388631567

In [73]:
get_probability(
    model, 
    [None, None, None, True, False], 
    [False, None, None, None, None]
)

0.04984794900000031

## Task3: Diagnosing
![task3](figures/task3.svg)

In [85]:
patientage_domain = ("0-30", "31-65", "65+")
ctscanresult_domain = ("Ischemic Stroke", "Hemmorraghic Stroke")
mriscanresult_domain = ("Ischemic Stroke", "Hemmorraghic Stroke")
stroketype_domain = ("Ischemic Stroke", "Hemmorraghic Stroke", "Stroke Mimic")
anticoagulants_domain = ("Used", "Not used")
mortality_domain = ("False", "True")
disability_domain = ("Negligible", "Moderate", "Severe")

patientage = pomegranate.DiscreteDistribution(
    {
        "0-30": 0.10,
        "31-65": 0.30,
        "65+": 0.60
    }
)

ctscanresult = pomegranate.DiscreteDistribution(
    {
        "Ischemic Stroke": 0.70,
        "Hemmorraghic Stroke": 0.30
    }
)

mriscanresult = pomegranate.DiscreteDistribution(
    {
        "Ischemic Stroke": 0.70,
        "Hemmorraghic Stroke": 0.30
    }
)

anticoagulants = pomegranate.DiscreteDistribution(
    {
        "Used": 0.5,
        "Not used": 0.5
    }
)

stroketype = [0.8, 0.5, 0.5, 0.0, 
              0.0, 0.4, 0.4, 0.9, 
              0.2, 0.1, 0.1, 0.1]
f = lambda i, j, k: stroketype[4 * k + 2 * i + j]
stroketype = [
    [x, y, z, f(i, j, k)]
        for k, z in enumerate(stroketype_domain)
            for i, x in enumerate(ctscanresult_domain)
                for j, y in enumerate(mriscanresult_domain)
]
stroketype = pomegranate.ConditionalProbabilityTable(
    stroketype, [ctscanresult, mriscanresult])


mortality = [0.28, 0.99, 0.1, 0.56, 0.58, 0.05, 
             0.72, 0.01, 0.9, 0.44, 0.42, 0.95]
f = lambda i, j, k: mortality[6 * k + 3 * j + i]
mortality = [
    [x, y, z, f(i, j, k)]
        for k, z in enumerate(mortality_domain)
            for j, y in enumerate(anticoagulants_domain)
                for i, x in enumerate(stroketype_domain)
]
mortality = pomegranate.ConditionalProbabilityTable(
    mortality, [stroketype, anticoagulants])


disability = [0.80, 0.70, 0.90, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 
              0.10, 0.20, 0.05, 0.30, 0.40, 0.30, 0.40, 0.20, 0.10, 
              0.10, 0.10, 0.05, 0.10, 0.10, 0.30, 0.30, 0.60, 0.80]
f = lambda i, j, k: disability[9 * k + 3 * j + i]
disability = [
    [x, y, z, f(i, j, k)]
        for k, z in enumerate(disability_domain)
            for j, y in enumerate(patientage_domain)
                for i, x in enumerate(stroketype_domain)
]
disability = pomegranate.ConditionalProbabilityTable(
    disability, [stroketype, patientage]
)

In [86]:
s1 = pomegranate.State(ctscanresult, name="CTScanResult")
s2 = pomegranate.State(mriscanresult, name="MRIScanResult")
s3 = pomegranate.State(patientage, name="PatientAge")
s4 = pomegranate.State(stroketype, name="StrokeType")
s5 = pomegranate.State(anticoagulants, name="Anticoagulants")
s6 = pomegranate.State(disability, name="Disability")
s7 = pomegranate.State(mortality, name="Mortality")

model = pomegranate.BayesianNetwork("Diagnosing")
model.add_states(s1, s2, s3, s4, s5, s6, s7)
model.add_transition(s1, s4)
model.add_transition(s2, s4)
model.add_transition(s3, s6)
model.add_transition(s4, s6)
model.add_transition(s4, s7)
model.add_transition(s5, s7)
model.bake()

In [87]:
# p(mortality = true | patientage = 0-30, ctscanresult = ischemic stroke)
get_probability(
    model, 
    [None, None, None, None, None, None, "True"], 
    ["Ischemic Stroke", None, "0-30", None, None, None, None]
)

0.5948499999999999

In [88]:
# p(disa = "severe" | patient=65+, mri=ischemic)
get_probability(
    model, 
    [None, None, None, None, None, "Severe", None], 
    [None, "Ischemic Stroke", "65+", None, None, None, None]
)

0.42100000000000004

In [89]:
# p(stroke=mimic | patient=65+, ct=Hemmo, mri=Ischemic)
get_probability(
    model, 
    [None, None, None, "Stroke Mimic", None, None, None], 
    ["Hemmorraghic Stroke", "Ischemic Stroke", "65+", None, None, None, None]
)

0.10000000000000045

In [90]:
# p(mortality=false | age=0-30, antico=used, stroke=mimic)
get_probability(
    model, 
    [None, None, None, None, None, None, "False"], 
    [None, None, "0-30", "Stroke Mimic", "Used", None, None]
)

0.1000000000000002

In [91]:
# p(age=0-30, ct=Is, mri=Hemmo, antico=Used, type=Mimic, disability=severe, mortality=false)
get_probability(
    model, 
    ["Ischemic Stroke", "Hemmorraghic Stroke", "0-30", "Stroke Mimic", "Used", "Severe", "False"], 
    [None, None, None, None, None, None, None]
)

5.2500000000000726e-06